In [ ]:
import numpy as np
import tensorflow as tf
import random

# Tải và tiền xử lý dữ liệu Fashion MNIST
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train / 255.0  # Chuẩn hóa
x_test = x_test / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

# Hàm tạo mô hình (tương tự create_model trong notebook)
def create_model(individual):
    num_hidden_layers, num_units, dropout_rate, learning_rate, batch_size = individual
    num_hidden_layers = int(num_hidden_layers)
    num_units = int(num_units)
    batch_size = int(batch_size)

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))

    for _ in range(num_hidden_layers):
        model.add(tf.keras.layers.Dense(num_units, activation='relu'))
        model.add(tf.keras.layers.Dropout(dropout_rate))

    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model, batch_size

# Hàm đánh giá (fitness function)
def evaluate_individual(individual):
    model, batch_size = create_model(individual)
    try:
        model.fit(
            x_train, y_train,
            batch_size=batch_size,
            epochs=5,
            validation_data=(x_test, y_test),
            verbose=0
        )
        _, val_accuracy = model.evaluate(x_test, y_test, verbose=0)
        return val_accuracy
    except:
        return 0.0  # Trả về fitness thấp nếu có lỗi

# Khởi tạo quần thể
def initialize_population(pop_size):
    population = []
    for _ in range(pop_size):
        num_hidden_layers = random.choice([1, 2, 3])
        num_units = random.choice([8, 16, 32])
        dropout_rate = random.uniform(0.1, 0.5)
        learning_rate = random.uniform(0.0001, 0.01)
        batch_size = random.choice([32, 64, 96, 128])
        population.append((num_hidden_layers, num_units, dropout_rate, learning_rate, batch_size))
    return population

# Lai ghép (crossover)
def crossover(parent1, parent2):
    w = random.random()
    new_hidden_layers = random.choice([parent1[0], parent2[0]])
    new_units = random.choice([parent1[1], parent2[1]])
    new_dropout = w * parent1[2] + (1 - w) * parent2[2]
    new_lr = w * parent1[3] + (1 - w) * parent2[3]
    new_batch_size = random.choice([parent1[4], parent2[4]])
    # Giới hạn giá trị
    new_dropout = max(0.1, min(0.5, new_dropout))
    new_lr = max(0.0001, min(0.01, new_lr))
    return (new_hidden_layers, new_units, new_dropout, new_lr, new_batch_size)

# Đột biến (mutation)
def mutate(individual, mutation_rate=0.1):
    num_hidden_layers, num_units, dropout_rate, learning_rate, batch_size = individual
    if random.random() < mutation_rate:
        num_hidden_layers = random.choice([1, 2, 3])
    if random.random() < mutation_rate:
        num_units = random.choice([8, 16, 32])
    if random.random() < mutation_rate:
        dropout_rate = dropout_rate + random.gauss(0, 0.05)
        dropout_rate = max(0.1, min(0.5, dropout_rate))
    if random.random() < mutation_rate:
        learning_rate = learning_rate + random.gauss(0, 0.001)
        learning_rate = max(0.0001, min(0.01, learning_rate))
    if random.random() < mutation_rate:
        batch_size = random.choice([32, 64, 96, 128])
    return (num_hidden_layers, num_units, dropout_rate, learning_rate, batch_size)

# Thuật toán di truyền
def genetic_algorithm(pop_size=20, n_generations=10, mutation_rate=0.1, elitism=0.1):
    population = initialize_population(pop_size)
    best_individuals = []

    for generation in range(n_generations):
        # Đánh giá fitness
        fitness_scores = [(ind, evaluate_individual(ind)) for ind in population]
        fitness_scores.sort(key=lambda x: x[1], reverse=True)

        # Lưu 3 cá thể tốt nhất
        best_individuals = fitness_scores[:3]

        # In thông tin thế hệ
        best_individual, best_fitness = fitness_scores[0]
        print(f"Generation {generation + 1}: Best Val Accuracy = {best_fitness:.4f}, "
              f"Hidden Layers = {int(best_individual[0])}, "
              f"Units = {int(best_individual[1])}, "
              f"Dropout = {best_individual[2]:.4f}, "
              f"LR = {best_individual[3]:.6f}, "
              f"Batch Size = {int(best_individual[4])}")

        # Elitism
        n_elite = int(elitism * pop_size)
        new_population = [ind for ind, _ in fitness_scores[:n_elite]]

        # Tạo cá thể mới
        while len(new_population) < pop_size:
            parents = random.sample(fitness_scores[:pop_size//2], 2)
            parent1, parent2 = parents[0][0], parents[1][0]
            child = crossover(parent1, parent2)
            child = mutate(child, mutation_rate)
            new_population.append(child)

        population = new_population

    return best_individuals

# Chạy thuật toán di truyền
best_three = genetic_algorithm(pop_size=20, n_generations=3, mutation_rate=0.1)
print("\nThree best parameter sets found:")
for i, (params, accuracy) in enumerate(best_three, 1):
    print(f"Set {i}: Val Accuracy = {accuracy:.4f}, "
          f"Hidden Layers = {int(params[0])}, "
          f"Units = {int(params[1])}, "
          f"Dropout = {params[2]:.4f}, "
          f"Learning Rate = {params[3]:.6f}, "
          f"Batch Size = {int(params[4])}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Generation 1: Best Val Accuracy = 0.8535, Hidden Layers = 1, Units = 32, Dropout = 0.1300, LR = 0.000992, Batch Size = 96
Generation 2: Best Val Accuracy = 0.8546, Hidden Layers = 1, Units = 32, Dropout = 0.1302, LR = 0.003943, Batch Size = 96
Generation 3: Best Val Accuracy = 0.8595, Hidden Layers = 1, Units = 32, Dropout = 0.1300, LR = 0.000992, Batch Size = 96


In [ ]:
# Bản cải tiến cho accuracy cao nhất (maybe?)

import numpy as np
import tensorflow as tf
import random
from sklearn.model_selection import train_test_split

# Tải và tiền xử lý dữ liệu Fashion MNIST
(x_train_full, y_train_full), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train_full = x_train_full / 255.0
x_test = x_test / 255.0
y_train_full = y_train_full.astype(np.int32)
y_test = y_test.astype(np.int32)

# Chia tập train thành train và validation (80/20)
x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, test_size=0.2, random_state=42)

# Hàm tạo mô hình
def create_model(individual):
    num_hidden_layers, num_units, dropout_rate, learning_rate, batch_size = individual
    num_hidden_layers = int(num_hidden_layers)
    num_units = int(num_units)
    batch_size = int(batch_size)

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))

    for _ in range(num_hidden_layers):
        model.add(tf.keras.layers.Dense(num_units, activation='relu'))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dropout(dropout_rate))

    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model, batch_size

# Hàm đánh giá (fitness function)
def evaluate_individual(individual):
    model, batch_size = create_model(individual)
    try:
        model.fit(
            x_train, y_train,
            batch_size=batch_size,
            epochs=10,
            validation_data=(x_val, y_val),
            callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)],
            verbose=0
        )
        _, val_accuracy = model.evaluate(x_val, y_val, verbose=0)
        return val_accuracy
    except:
        return 0.0

# Khởi tạo quần thể
def initialize_population(pop_size):
    population = []
    for _ in range(pop_size):
        num_hidden_layers = random.choice([1, 2, 3, 4])
        num_units = random.choice([32, 64, 128, 256])
        dropout_rate = random.uniform(0.1, 0.3)
        learning_rate = random.uniform(0.0001, 0.01)
        batch_size = random.choice([32, 64, 128, 256])
        population.append((num_hidden_layers, num_units, dropout_rate, learning_rate, batch_size))
    return population

# Lai ghép (crossover)
def crossover(parent1, parent2):
    w = random.random()
    new_hidden_layers = random.choice([parent1[0], parent2[0]])
    new_units = random.choice([parent1[1], parent2[1]])
    new_dropout = w * parent1[2] + (1 - w) * parent2[2]
    new_lr = w * parent1[3] + (1 - w) * parent2[3]
    new_batch_size = random.choice([parent1[4], parent2[4]])
    new_dropout = max(0.1, min(0.3, new_dropout))
    new_lr = max(0.0001, min(0.01, new_lr))
    return (new_hidden_layers, new_units, new_dropout, new_lr, new_batch_size)

# Đột biến (mutation)
def mutate(individual, mutation_rate=0.1):
    num_hidden_layers, num_units, dropout_rate, learning_rate, batch_size = individual
    if random.random() < mutation_rate:
        num_hidden_layers = random.choice([1, 2, 3, 4])
    if random.random() < mutation_rate:
        num_units = random.choice([32, 64, 128, 256])
    if random.random() < mutation_rate:
        dropout_rate = dropout_rate + random.gauss(0, 0.05)
        dropout_rate = max(0.1, min(0.3, dropout_rate))
    if random.random() < mutation_rate:
        learning_rate = learning_rate + random.gauss(0, 0.001)
        learning_rate = max(0.0001, min(0.01, learning_rate))
    if random.random() < mutation_rate:
        batch_size = random.choice([32, 64, 128, 256])
    return (num_hidden_layers, num_units, dropout_rate, learning_rate, batch_size)

# Thuật toán di truyền
def genetic_algorithm(pop_size=30, n_generations=15, mutation_rate=0.1, elitism=0.1):
    population = initialize_population(pop_size)
    best_individuals = []

    for generation in range(n_generations):
        fitness_scores = [(ind, evaluate_individual(ind)) for ind in population]
        fitness_scores.sort(key=lambda x: x[1], reverse=True)

        best_individuals = fitness_scores[:3]

        best_individual, best_fitness = fitness_scores[0]
        print(f"Generation {generation + 1}: Best Val Accuracy = {best_fitness:.4f}, "
              f"Hidden Layers = {int(best_individual[0])}, "
              f"Units = {int(best_individual[1])}, "
              f"Dropout = {best_individual[2]:.4f}, "
              f"LR = {best_individual[3]:.6f}, "
              f"Batch Size = {int(best_individual[4])}")

        n_elite = int(elitism * pop_size)
        new_population = [ind for ind, _ in fitness_scores[:n_elite]]

        while len(new_population) < pop_size:
            parents = random.sample(fitness_scores[:pop_size//2], 2)
            parent1, parent2 = parents[0][0], parents[1][0]
            child = crossover(parent1, parent2)
            child = mutate(child, mutation_rate)
            new_population.append(child)

        population = new_population

    return best_individuals

# Chạy thuật toán di truyền
best_three = genetic_algorithm(pop_size=30, n_generations=5, mutation_rate=0.1)

# In 3 bộ siêu tham số tốt nhất
print("\nThree best parameter sets found (based on validation accuracy):")
for i, (params, accuracy) in enumerate(best_three, 1):
    print(f"Set {i}: Val Accuracy = {accuracy:.4f}, "
          f"Hidden Layers = {int(params[0])}, "
          f"Units = {int(params[1])}, "
          f"Dropout = {params[2]:.4f}, "
          f"Learning Rate = {params[3]:.6f}, "
          f"Batch Size = {int(params[4])}")

# Đánh giá lại mô hình tốt nhất trên tập test
best_params = best_three[0][0]
model, batch_size = create_model(best_params)
history = model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=20,
    validation_data=(x_val, y_val),
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)],
    verbose=1
)
_, test_accuracy = model.evaluate(x_test, y_test, verbose=0) # model.get_weights() để lấy các tham số
print(f"\nFinal Test Accuracy with best parameters: {test_accuracy:.4f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Generation 1: Best Val Accuracy = 0.8848, Hidden Layers = 2, Units = 64, Dropout = 0.1064, LR = 0.001531, Batch Size = 64
Generation 2: Best Val Accuracy = 0.8832, Hidden Layers = 2, Units = 256, Dropout = 0.1109, LR = 0.000858, Batch Size = 256
Generation 3: Best Val Accuracy = 0.8863, Hidden Layers = 2, Units = 256, Dropout = 0.1109, LR = 0.000858, Batch Size = 256
Generation 4: Best Val Accuracy = 0.8863, Hidden Layers = 2, Units = 256, Dropout = 0.1109, LR = 0.000858, Batch Size = 256
Generation 5: Best Val Accuracy = 0.8812, Hidden Layers = 2, Units = 256, Dropout = 0.2242, LR = 0.003323, Batch Size = 64

Three best parameter sets found (based on validation accuracy):
Set 1: Val Accuracy = 0.8812, Hidden Layers = 2, Units = 256, Dropout = 0.2242, Learning Rate = 0.003323, Batch Size = 64
Set 2: Val Accuracy = 0.8802, Hidden Layers = 2, Units = 256, Dropout = 0.2179, Learning Rate = 0.003079, Batch Size = 256
Set 3: Val Accuracy = 0.8800, Hidden Layers = 2, Units = 256, Dropout = 0